In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sa
from comprl.server.data.sql_backend import User, Game, get_user_pair_statistics

# 24h Tournament

In [ ]:
# Adjust path to database here, then let everything run
database_path = "/home/felixwidmaier/projects/comprl/hockey_backup_2025-02-24.db"
engine = sa.create_engine(f"sqlite:///{database_path}")

In [ ]:
with sa.orm.Session(engine) as session:
    stmt = (
        sa.select(User.username, sa.func.count(Game.id).label("count"))
        .join(Game, sa.or_(Game.user1 == User.user_id, Game.user2 == User.user_id))
        .group_by(User.username)
        .order_by(sa.desc("count"))
    )
    num_games_per_user = session.execute(stmt).all()

In [ ]:
usernames, num_games = zip(*num_games_per_user)
fig, ax = plt.subplots(1,1, figsize=(16,8))
plt.xticks(rotation='vertical')
plt.bar(x=range(len(num_games_per_user)),height=num_games, log=True, tick_label=usernames)
plt.ylabel("Number of games played")
plt.grid(axis="y")
fig.savefig("Num_games-played.png", dpi=200,bbox_inches="tight")

In [ ]:
# get ranked users
with sa.orm.Session(engine) as session:
    stmt = sa.select(User).order_by((User.mu - User.sigma).desc())
    ranking = session.scalars(stmt).all()
n_users = len(ranking)

In [ ]:
# get pairwise game statistics
with sa.orm.Session(engine) as session:
    user_pairwise_stats = get_user_pair_statistics(session, [user.user_id for user in ranking])

In [ ]:
lcb = np.asarray([user.mu - user.sigma for user in ranking])

In [ ]:
results = np.zeros((n_users, n_users))
for i, user1 in enumerate(ranking):
    for j, user2 in enumerate(ranking):
        if user1 == user2:
            continue

        entry = user_pairwise_stats[(user1.user_id, user2.user_id)]
        n_total = entry['wins'] + entry['losses'] + entry['draws']
        if n_total > 0:
            results[i][j] = entry['wins'] / n_total

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,18))
sns.heatmap(
    results,
    yticklabels=[user.username for user in ranking],
    xticklabels=[user.username for user in ranking],
    cmap='inferno',
)
ax.set_title('Win Rate')
fig.savefig("Win-matrix.png", dpi=200, bbox_inches="tight")

In [ ]:
# Only-win-loose ratio

In [ ]:
results = np.zeros((n_users, n_users))
for i, user1 in enumerate(ranking):
    for j, user2 in enumerate(ranking):
        if user1 == user2:
            continue

        entry = user_pairwise_stats[(user1.user_id, user2.user_id)]
        n_total = entry['wins'] + entry['losses']
        if n_total > 0:
            results[i][j] = entry['wins'] / n_total

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,18))
sns.heatmap(
    results,
    yticklabels=[user.username for user in ranking],
    xticklabels=[user.username for user in ranking],
    cmap='inferno',
)
ax.set_title('Win vs Lose - Rate')
fig.savefig("Win-lose-matrix.png", dpi=200, bbox_inches="tight")

In [ ]:
# Reduce ranking to top users
n_users = 15
ranking = ranking[:n_users]

In [ ]:
results = np.zeros((n_users, n_users))
for i, user1 in enumerate(ranking):
    for j, user2 in enumerate(ranking):
        if user1 == user2:
            continue

        entry = user_pairwise_stats[(user1.user_id, user2.user_id)]
        n_total = entry['wins'] + entry['losses']
        if n_total > 0:
            results[i][j] = entry['wins'] / n_total

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,7))
sns.heatmap(
    results,
    yticklabels=[user.username for user in ranking],
    xticklabels=[user.username for user in ranking],
    cmap='inferno',
    square=True,
)
ax.set_title('Win vs Lose - Rate')
fig.savefig("Win-lose-matrix-Top10.png", dpi=200, bbox_inches="tight")

In [ ]:
results = np.zeros((n_users, n_users))
for i, user1 in enumerate(ranking):
    for j, user2 in enumerate(ranking):
        if user1 == user2:
            continue
            
        entry = user_pairwise_stats[(user1.user_id, user2.user_id)]
        results[i][j] = np.log10(entry['wins'] + entry['losses'] +  entry['draws']+1)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,7))
sns.heatmap(
    results,
    yticklabels=[user.username for user in ranking],
    xticklabels=[user.username for user in ranking],
    cmap='inferno',
    square=True,
)
ax.set_title('Number of games played (log10)')
fig.savefig("games-played-Top10.png", dpi=200, bbox_inches="tight")